In [1]:
import os
mydir='/media/garyfan/My Passport/h2s10-LM/'

In [12]:
from ase import Atoms, io
from ase import units
from math import cos, sin, pi
from ase.calculators.tip4p import TIP4P, epsilon0, sigma0, rOH, angleHOH
from ase.constraints import FixInternals
from ase.neb import NEB
from ase.optimize import MDMin, BFGS,LBFGS
from ase.optimize.basin_mul import BasinHoppingm
import pandas as pd
import numpy as np
from tools.h2s_cluster import h2scluster
from tools.h2s_cluster import add_const
import os
import argparse
import math 
from ase.optimize import LBFGS, FIRE


In [3]:
from os import listdir
def list_of_files(dir_name, suffix):
    return [f for f in listdir(dir_name) if f.endswith('.' + suffix)]
# load minima 
#ftraj = args.lm 

minima = list_of_files(mydir, 'xyz')
nminima = len(minima)
nminima

200

In [4]:
LM = {}
for mini in minima:
    LM[mini[:-4]]= io.read(mydir+mini)
for lm in LM.values():
    add_const(lm)


In [5]:

sep = math.floor(nminima/50)
minima=minima[-1::-sep]
nminima = len(minima)
print('# minima used: ', nminima)
LM = {}
for mini in minima:
    LM[mini[:-4]]= io.read(mydir+mini)
for lm in LM.values():
    add_const(lm)


# minima used:  50


In [6]:
for lm in LM.values():
    dyn=LBFGS(lm)
    dyn.run(fmax=0.05)
    

       Step     Time          Energy         fmax
LBFGS:    0 00:21:03       -1.267430        0.0643
LBFGS:    1 00:21:03       -1.267802        0.0642
LBFGS:    2 00:21:03       -1.302150        0.0533
LBFGS:    3 00:21:03       -1.302490        0.0562
LBFGS:    4 00:21:03       -1.303694        0.0610
LBFGS:    5 00:21:03       -1.306183        0.0645
LBFGS:    6 00:21:04       -1.311465        0.0642
LBFGS:    7 00:21:04       -1.319032        0.0585
LBFGS:    8 00:21:04       -1.325726        0.0462
       Step     Time          Energy         fmax
LBFGS:    0 00:21:04       -1.338075        0.0936
LBFGS:    1 00:21:04       -1.338706        0.0926
LBFGS:    2 00:21:04       -1.401706        0.0645
LBFGS:    3 00:21:04       -1.404559        0.0686
LBFGS:    4 00:21:04       -1.405021        0.0686
LBFGS:    5 00:21:04       -1.410502        0.0764
LBFGS:    6 00:21:04       -1.418109        0.0804
LBFGS:    7 00:21:04       -1.431311        0.0630
LBFGS:    8 00:21:04       -1.440

       Step     Time          Energy         fmax
LBFGS:    0 00:21:09       -1.338221        0.0903
LBFGS:    1 00:21:09       -1.338654        0.0898
LBFGS:    2 00:21:09       -1.368138        0.0836
LBFGS:    3 00:21:09       -1.369226        0.0725
LBFGS:    4 00:21:09       -1.369528        0.0659
LBFGS:    5 00:21:09       -1.370843        0.0508
LBFGS:    6 00:21:09       -1.374163        0.0297
       Step     Time          Energy         fmax
LBFGS:    0 00:21:09       -1.303296        0.0600
LBFGS:    1 00:21:09       -1.303649        0.0596
LBFGS:    2 00:21:09       -1.336522        0.0922
LBFGS:    3 00:21:09       -1.337836        0.0772
LBFGS:    4 00:21:09       -1.338259        0.0711
LBFGS:    5 00:21:09       -1.342341        0.0389
       Step     Time          Energy         fmax
LBFGS:    0 00:21:09       -1.277786        0.0560
LBFGS:    1 00:21:09       -1.278168        0.0558
LBFGS:    2 00:21:09       -1.318919        0.0498
       Step     Time          Ener

       Step     Time          Energy         fmax
LBFGS:    0 00:21:13       -1.269332        0.0686
LBFGS:    1 00:21:13       -1.269800        0.0682
LBFGS:    2 00:21:13       -1.300691        0.0401
       Step     Time          Energy         fmax
LBFGS:    0 00:21:13       -1.325286        0.1416
LBFGS:    1 00:21:13       -1.326080        0.1411
LBFGS:    2 00:21:13       -1.385610        0.0517
LBFGS:    3 00:21:14       -1.396252        0.1016
LBFGS:    4 00:21:14       -1.399263        0.0634
LBFGS:    5 00:21:14       -1.399740        0.0633
LBFGS:    6 00:21:14       -1.423478        0.0495
       Step     Time          Energy         fmax
LBFGS:    0 00:21:14       -1.475454        0.0738
LBFGS:    1 00:21:14       -1.475803        0.0732
LBFGS:    2 00:21:14       -1.499597        0.0367


In [15]:

import itertools
transition=pd.DataFrame(columns=['m1', 'm2', 'Ed', 'images'])
lmlist=LM.keys()
comb = list(itertools.product(lmlist, lmlist))
comb =[list(e) for e in set(frozenset(d) for d in comb)]
dcomb = []
for i in comb:
    if len(i)==2:
        dcomb.append(i)

dcomb=np.array(dcomb)
transition['m1'] = dcomb[:, 0]
#transition['m1']= transition['m1'].apply(lambda x: int(x))
transition['m2'] = dcomb[:, 1]
#transition['m2']= transition['m2'].apply(lambda x: int(x))
transition = transition[transition['m1']!=transition['m2']]
print(transition.head())
npdf=transition.values



########### neb 
import signal
class TimedOutExc(Exception):
    pass

def deadline(timeout, *args):
    def decorate(f):
        def handler(signum, frame):
            raise TimedOutExc()

        def new_f(*args):
            signal.signal(signal.SIGALRM, handler)
            signal.alarm(timeout)
            return f(*args)
            signal.alarm(0)

        new_f.__name__ = f.__name__
        return new_f
    return decorate

@deadline(50)
def optimize(neb="", name=''):
    optimizer = FIRE(neb, trajectory=name+'.traj')
    optimizer.run(fmax=100000)
    optimizer.run(fmax=50000)
    optimizer.run(fmax=10000)
    optimizer.run(fmax=5000)
    optimizer.run(fmax=1000)
    optimizer.run(fmax=500)
    optimizer.run(fmax=100)
    optimizer.run(fmax=50)
    optimizer.run(fmax=10)
    optimizer.run(fmax=0.5)
    optimizer.run(fmax=0.4)


    m1   m2   Ed images
0  173    4  NaN    NaN
1  164  193  NaN    NaN
2   21   43  NaN    NaN
3   33   43  NaN    NaN
4  131   76  NaN    NaN


In [14]:
min1='63'
min2='195'
images=[LM[min1]]
images += [LM[min1].copy() for i in range(3)]
images += [LM[min2]]
for image in images:
    add_const(image)
neb = NEB(images, parallel=True) 
neb.interpolate()
optimize(neb, "check")

      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 00:24:27  4816394.033795* 40385774.9336
FIRE:    1 00:24:27  1186985.903826* 7272476.2731
FIRE:    2 00:24:27   596957.554593* 4603817.1661
FIRE:    3 00:24:27   222528.318020* 1511061.9335
FIRE:    4 00:24:27   148980.171406*  538590.4931
FIRE:    5 00:24:28    94767.324622*  283899.2176
FIRE:    6 00:24:28    57274.576104*  202202.2959
FIRE:    7 00:24:28    32456.638791*  120414.8695
FIRE:    8 00:24:28    17164.754305*   61236.2066
FIRE:    9 00:24:28     8615.136767*   27558.4485
FIRE:   10 00:24:29     4240.455330*   11394.4317
FIRE:   11 00:24:29     2141.461525*    4484.8617
FIRE:   12 00:24:29     1502.081572*    2899.6490
FIRE:   13 00:24:30     1515.771128*    3884.8290
FIRE:   14 00:24:30     1500.742486*    4518.4160
FIRE:   15 00:24:30     1409.864668*    4481.7366
FIRE:   16 00:24:30     1242.040747*    3799.8310
FIRE:   17 00:24:30     1030.823389*    2760.1936
F

FIRE:  162 00:24:58       -0.386121*       0.3956
FIRE:  163 00:24:58       -0.407354*       0.3868
FIRE:  164 00:24:58       -0.428989*       0.3810
FIRE:  165 00:24:59       -0.451053*       0.3772
FIRE:  166 00:24:59       -0.473537*       0.3747
FIRE:  167 00:24:59       -0.494874*       0.4268
FIRE:  168 00:24:59       -0.496006*       0.3710
FIRE:  169 00:24:59       -0.498796*       0.3706
FIRE:  170 00:25:00       -0.504384*       0.3699
FIRE:  171 00:25:00       -0.512773*       0.3687
FIRE:  172 00:25:00       -0.511562*       0.4173
FIRE:  173 00:25:00       -0.512938*       0.4171
FIRE:  174 00:25:00       -0.515673*       0.4166
FIRE:  175 00:25:01       -0.519691*       0.4158
FIRE:  176 00:25:01       -0.523377*       0.3656
FIRE:  177 00:25:01       -0.523546*       0.3656
FIRE:  178 00:25:01       -0.523891*       0.3655
FIRE:  179 00:25:01       -0.524407*       0.3654
FIRE:  180 00:25:01       -0.524365*       0.4145
FIRE:  181 00:25:02       -0.524442*       0.4145


TimedOutExc: 

In [ ]:

from ase.neb import NEBTools
from ase.io import read
def saddlepoint(min1, min2, name):
    images=[LM[min1]]
    images += [LM[min1].copy() for i in range(3)]
    images += [LM[min2]]
    for image in images:
        add_const(image)
    neb = NEB(images, parallel=True) 
    try:
        neb.interpolate()
    except:
        try:
            print("try idpp interpolate")
            neb.interpolate(method="idpp")
        except:
            return
    optimize(neb, name)
ts = '/media/garyfan/My Passport/h2s10_ts'
###os.mkdir(ts) 
records=0
for line in npdf:
    records+=1
    try:
        line[3]=line[0]+'_'+line[1]
        saddlepoint(line[0], line[1], ts+ "/"+line[3])
    except:
        print(line[0], line[1], "fail")
        continue
transition=pd.DataFrame(npdf, columns=['m1', 'm2', 'Ed', 'images'])
transition.to_csv(ts+'.csv', index=False)



      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 00:25:27 2466255053.145521* 52905786795.2528
FIRE:    1 00:25:27 57109230.591567* 467239798.8375
FIRE:    2 00:25:27 115338061.682406* 1443799177.9500
FIRE:    3 00:25:27  7279869.357264* 37130261.1201
FIRE:    4 00:25:28  5709116.722870* 43697749.8793
FIRE:    5 00:25:28  1651045.547440* 7155242.5741
FIRE:    6 00:25:28   688184.096019* 3560510.1859
FIRE:    7 00:25:28   290545.909017* 1937815.3141
FIRE:    8 00:25:28   107915.684488*  700199.9803
FIRE:    9 00:25:28    37497.272238*  198915.2044
FIRE:   10 00:25:29    15635.216727*   55014.2258
FIRE:   11 00:25:29     8786.551064*   33197.0637
FIRE:   12 00:25:29     6611.872446*   24668.5999
FIRE:   13 00:25:29     5824.698791*   16296.3792
FIRE:   14 00:25:30     5497.427690*   11473.2062
FIRE:   15 00:25:30     5371.969475*   16897.4216
FIRE:   16 00:25:30     5170.121395*   20462.7938
FIRE:   17 00:25:30     4618.469568*  

FIRE:    1 00:26:00 150972446.347666* 2549256246.9907
FIRE:    2 00:26:00  3487807.590285* 42505391.8134
FIRE:    3 00:26:01   199694.734096* 1840624.5235
FIRE:    4 00:26:01    20265.604405*  142812.7711
FIRE:    5 00:26:01     3277.239514*   16425.4162
FIRE:    6 00:26:01      959.152900*    2503.7572
FIRE:    7 00:26:02      500.888323*    1938.3655
FIRE:    8 00:26:02      338.903880*    1477.7513
FIRE:    9 00:26:02      253.384668*     991.8307
FIRE:   10 00:26:03      225.868247*     607.3186
FIRE:   11 00:26:03      258.778888*    1157.7018
FIRE:   12 00:26:03      332.837740*    1878.3384
FIRE:   13 00:26:03      102.262996*     353.8993
FIRE:   14 00:26:03       58.157775*     153.0189
FIRE:   15 00:26:04       42.187450*      98.7056
FIRE:   16 00:26:04       29.798372*      65.6383
FIRE:   17 00:26:04       20.940912*      42.3681
FIRE:   18 00:26:05       15.413417*      28.5018
FIRE:   19 00:26:05       13.670600*      22.8841
FIRE:   20 00:26:05       12.484455*      17.

FIRE:    8 00:26:34      408.963052*     860.4245
FIRE:    9 00:26:34      367.640504*     657.1364
FIRE:   10 00:26:34      333.736804*     811.8737
FIRE:   11 00:26:34      291.682021*     835.0689
FIRE:   12 00:26:35      239.267231*     710.7188
FIRE:   13 00:26:35      186.476733*     510.2552
FIRE:   14 00:26:35      142.829703*     318.5242
FIRE:   15 00:26:35      111.411003*     180.2092
FIRE:   16 00:26:36       89.855674*     124.0311
FIRE:   17 00:26:36       74.235661*     116.1348
FIRE:   18 00:26:36       61.720190*     102.0346
FIRE:   19 00:26:36       51.000433*      84.5144
FIRE:   20 00:26:36       41.681473*      66.7397
FIRE:   21 00:26:37       33.723894*      54.1232
FIRE:   22 00:26:37       27.101577*      41.1767
FIRE:   23 00:26:37       21.728078*      29.5103
FIRE:   24 00:26:37       18.021195*      20.7480
FIRE:   25 00:26:37       16.635342*      15.2666
FIRE:   26 00:26:37       15.263466*      12.3380
FIRE:   27 00:26:38       13.881745*      11.1132


FIRE:   55 00:27:07       -0.301950*       0.4187
FIRE:   56 00:27:07       -0.321392*       0.4090
FIRE:   57 00:27:07       -0.340509*       0.3967
      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 00:27:08   684942.188488* 8130199.6970
FIRE:    1 00:27:08    45478.935398*  291622.2648
FIRE:    2 00:27:08    17599.939937*   36828.9937
FIRE:    3 00:27:08    14056.665245*   32950.8173
FIRE:    4 00:27:09    11614.035273*   25493.5056
FIRE:    5 00:27:09     9494.601869*   17589.6936
FIRE:    6 00:27:09     8339.997726*   11935.2113
FIRE:    7 00:27:09     8384.231064*   17884.7114
FIRE:    8 00:27:09     9047.803417*   23395.5757
FIRE:    9 00:27:10     9260.555306*   25937.1968
FIRE:   10 00:27:10     8320.736360*   24074.7764
FIRE:   11 00:27:10     6438.027471*   18756.3424
FIRE:   12 00:27:10     4360.180674*   12495.7634
FIRE:   13 00:27:10     2672.531775*    7328.9034
FIRE:   14 00:27:11     1544.073977*    3909.5154
FI

FIRE:   24 00:27:38        6.784143*       4.3788
FIRE:   25 00:27:38        5.937669*       4.0434
FIRE:   26 00:27:39        5.168488*       3.5550
FIRE:   27 00:27:39        4.474604*       2.9830
FIRE:   28 00:27:39        3.858858*       2.4035
FIRE:   29 00:27:39        3.322834*       1.8758
FIRE:   30 00:27:39        2.863431*       1.4877
FIRE:   31 00:27:39        2.472355*       1.1770
FIRE:   32 00:27:40        2.139301*       0.9283
FIRE:   33 00:27:40        1.853978*       0.7492
FIRE:   34 00:27:40        1.607571*       0.7013
FIRE:   35 00:27:40        1.393233*       0.6426
FIRE:   36 00:27:40        1.205521*       0.5765
FIRE:   37 00:27:40        1.040277*       0.5089
FIRE:   38 00:27:41        0.912544*       0.4909
FIRE:   39 00:27:41        0.808771*       0.4311
FIRE:   40 00:27:41        0.714661*       0.3941
      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 00:27:41   617689.691888* 2993124.1776
FI

...

FIRE:   35 00:34:02        4.962976*       6.2927
FIRE:   36 00:34:02        4.157151*       4.4728
FIRE:   37 00:34:02        3.586076*       3.0841
FIRE:   38 00:34:03        3.084926*       2.5171
FIRE:   39 00:34:03        2.647324*       2.0842
FIRE:   40 00:34:03        2.268044*       1.6898
FIRE:   41 00:34:03        1.942146*       1.3390
FIRE:   42 00:34:03        1.666054*       1.1132
FIRE:   43 00:34:03        1.428748*       1.0240
FIRE:   44 00:34:04        1.229639*       0.9385
FIRE:   45 00:34:04        1.061515*       0.8488
FIRE:   46 00:34:04        0.919327*       0.7597
FIRE:   47 00:34:04        0.798558*       0.6737
FIRE:   48 00:34:04        0.695231*       0.5855
FIRE:   49 00:34:05        0.605993*       0.5196
FIRE:   50 00:34:05        0.528119*       0.4970
FIRE:   51 00:34:05        0.459458*       0.4696
FIRE:   52 00:34:05        0.398350*       0.4392
FIRE:   53 00:34:05        0.343535*       0.4168
FIRE:   54 00:34:06        0.294060*       0.3965


      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 00:34:34      199.417214*     698.0839
      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 00:34:34      199.417214*     698.0839
      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 00:34:34      199.417214*     698.0839
      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 00:34:35      199.417214*     698.0839
      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 00:34:35      199.417214*     698.0839
FIRE:    1 00:34:35       91.936358*     188.1006
FIRE:    2 00:34:35       59.188825*     109.6543
FIRE:    3 00:34:35       46.519242*      64.1768
FIRE:    4 00:34:36       40.160347*      37.2280
FIRE:    5 00:34:36       35.317691*      41.4625
FIRE:    6

FIRE:   59 00:35:04        0.651768*       0.4573
FIRE:   60 00:35:05        0.601607*       0.4457
FIRE:   61 00:35:05        0.553696*       0.4360
FIRE:   62 00:35:05        0.507817*       0.4278
FIRE:   63 00:35:05        0.463820*       0.4208
FIRE:   64 00:35:05        0.421594*       0.4149
FIRE:   65 00:35:05        0.381055*       0.4097
FIRE:   66 00:35:06        0.342126*       0.4054
FIRE:   67 00:35:06        0.304740*       0.4016
FIRE:   68 00:35:06        0.268827*       0.3982
      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 00:35:06  5730968.792606* 82824107.1142
FIRE:    1 00:35:06   210734.714228* 2283990.6564
FIRE:    2 00:35:07    15713.575774*  130928.6422
FIRE:    3 00:35:07     2036.776473*   11808.1461
FIRE:    4 00:35:07      754.998941*    1267.0617
FIRE:    5 00:35:08      694.134900*    1493.3273
FIRE:    6 00:35:08      654.611769*    1955.1180
FIRE:    7 00:35:08      741.036611*    2220.3937
F

FIRE:   97 00:35:38        0.717829*       1.0200
FIRE:   98 00:35:38        0.695402*       1.1088
FIRE:   99 00:35:38        0.673104*       1.1799
FIRE:  100 00:35:38        0.650880*       1.2239
FIRE:  101 00:35:38        0.628692*       1.2375
FIRE:  102 00:35:39        0.624936*       1.4231
FIRE:  103 00:35:39        0.638005*       1.3707
FIRE:  104 00:35:39        0.643893*       1.2712
FIRE:  105 00:35:39        0.644183*       1.1295
FIRE:  106 00:35:39        0.640784*       0.9582
FIRE:  107 00:35:40        0.635419*       0.7749
FIRE:  108 00:35:40        0.629336*       0.8663
FIRE:  109 00:35:40        0.622991*       0.9668
FIRE:  110 00:35:40        0.615947*       1.0598
FIRE:  111 00:35:40        0.607339*       1.1384
FIRE:  112 00:35:41        0.596518*       1.1963
FIRE:  113 00:35:41        0.582109*       1.2281
FIRE:  114 00:35:41        0.562986*       1.2319
FIRE:  115 00:35:41        0.538526*       1.2074
FIRE:  116 00:35:41        0.508769*       1.1562


FIRE:   30 00:36:09      347.583725*     380.1630
FIRE:   31 00:36:09      301.360051*     295.8459
FIRE:   32 00:36:10      261.397610*     267.6575
FIRE:   33 00:36:10      225.709537*     231.5167
FIRE:   34 00:36:10      192.980983*     204.8707
FIRE:   35 00:36:10      162.812846*     175.5052
FIRE:   36 00:36:10      135.456320*     143.3449
FIRE:   37 00:36:11      111.378519*     112.4539
FIRE:   38 00:36:11       90.876316*      85.5958
FIRE:   39 00:36:11       73.923033*      63.9093
FIRE:   40 00:36:11       63.113315*      48.9811
FIRE:   41 00:36:12       55.202228*      36.2877
FIRE:   42 00:36:12       48.457470*      30.0386
FIRE:   43 00:36:12       42.562869*      27.5747
FIRE:   44 00:36:12       37.320130*      24.5199
FIRE:   45 00:36:12       32.621516*      21.1088
FIRE:   46 00:36:13       28.414733*      17.6254
FIRE:   47 00:36:13       24.672196*      14.3256
FIRE:   48 00:36:13       21.371243*      11.3896
FIRE:   49 00:36:13       18.486341*       9.8234
